In [ ]:
print("Hello, World!!!!")

In [ ]:
import os
# Set working directory to project root so relative paths resolve consistently
os.chdir(r"D:\Sai Teja Honours\GenmedAssist")
print('CWD set to', os.getcwd())

In [ ]:
%pwd

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

def load_pdf_files(data):
    pdf_files = Path(data).glob("*.pdf")
    documents = []
    for pdf in pdf_files:
        loader = PyPDFLoader(str(pdf))
        documents.extend(loader.load())
    return documents

In [ ]:
extracted_data = load_pdf_files("data")

In [ ]:
len(extracted_data)

In [ ]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(documents: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only the original page_content and a metadata dict with 'source'.
    """
    minimal_docs: List[Document] = []
    for doc in documents:
        src = doc.metadata.get("source") if isinstance(doc.metadata, dict) else None
        minimal_docs.append(Document(page_content=doc.page_content, metadata={"source": src}))
    return minimal_docs

In [ ]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

# --- Clean Function ---
def clean_text(text):
    """Remove unwanted characters and normalize spacing."""
    text = text.replace('\n', ' ')               # Replace newlines with spaces
    text = re.sub(r'\s+', ' ', text).strip()     # Remove multiple spaces
    return text

# --- Split + Clean Function ---
def text_split(minimal_docs):
    """Clean and split documents into smaller chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )

    cleaned_docs = []
    for doc in minimal_docs:
        doc.page_content = clean_text(doc.page_content)
        cleaned_docs.append(doc)

    # Split the cleaned docs
    texts_chunk = text_splitter.split_documents(cleaned_docs)

    # Clean each chunk again (safety pass)
    for chunk in texts_chunk:
        chunk.page_content = clean_text(chunk.page_content)

    # Print all cleaned chunks before embedding
    print("\n📄 --- Cleaned Text Chunks ---\n")
    for i, chunk in enumerate(texts_chunk, 1):
        print(f"Chunk {i}:")
        print(chunk.page_content)
        print("-" * 80)

    return texts_chunkfrom langchain.text_splitter import RecursiveCharacterTextSplitter
import re

# --- Clean Function ---
def clean_text(text):
    """Remove unwanted characters and normalize spacing."""
    text = text.replace('\n', ' ')               # Replace newlines with spaces
    text = re.sub(r'\s+', ' ', text).strip()     # Remove multiple spaces
    return text

# --- Split + Clean Function ---
def text_split(minimal_docs):
    """Clean and split documents into smaller chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )

    cleaned_docs = []
    for doc in minimal_docs:
        doc.page_content = clean_text(doc.page_content)
        cleaned_docs.append(doc)

    # Split the cleaned docs
    texts_chunk = text_splitter.split_documents(cleaned_docs)

    # Clean each chunk again (safety pass)
    for chunk in texts_chunk:
        chunk.page_content = clean_text(chunk.page_content)

    # Print all cleaned chunks before embedding
    print("\n📄 --- Cleaned Text Chunks ---\n")
    for i, chunk in enumerate(texts_chunk, 1):
        print(f"Chunk {i}:")
        print(chunk.page_content)
        print("-" * 80)

    return texts_chunk

In [ ]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

In [ ]:
texts_chunk

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

In [ ]:
embedding

In [ ]:
vector = embedding.embed_query("Hello world")
vector

In [ ]:
print( "Vector length:", len(vector))

In [ ]:

print( "Vector length:", len(vector))

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:

from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
pc

In [ ]:
from pinecone import ServerlessSpec 

index_name = "genmedassist"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
ques = retriever.invoke("what are white blood cells ?")
ques